In [1]:
'''
Objective:
•
Apply the Support Vector Machines (SVM) algorithm to analyze a given dataset containing information about the population density and income levels of residents in a suburban locality, aiming to provide insights into the viability of investing in the area for a construction firm.
Constraints:
•
Ensure accurate preprocessing of the dataset, including handling missing values, scaling features, and selecting appropriate kernel functions, to prepare the data for SVM model training while maintaining data integrity and quality.
•
Conduct thorough analysis and interpretation of SVM model results, considering factors such as classification accuracy, precision, recall, and F1-score, to assess the potential profitability and risk associated with investing in the suburban locality.
Benefits/Impact:
•
The SVM algorithm provides valuable insights into the population density and income levels of residents in the suburban locality, aiding the construction firm in making informed decisions regarding infrastructure development and property sales strategies.
•
By evaluating the viability of investing in the area based on SVM model insights, the construction firm can mitigate potential losses, optimize resource allocation, and capitalize on opportunities for profitable investments, ultimately enhancing business sustainability and financial performance.


'''


import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from feature_engine.outliers import Winsorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
import joblib
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [19]:
salary = pd.read_csv('SalaryData_train.csv')
salary

age          workclass    education  educationno        maritalstatus  \
0       39          State-gov    Bachelors           13        Never-married   
1       50   Self-emp-not-inc    Bachelors           13   Married-civ-spouse   
2       38            Private      HS-grad            9             Divorced   
3       53            Private         11th            7   Married-civ-spouse   
4       28            Private    Bachelors           13   Married-civ-spouse   
...    ...                ...          ...          ...                  ...   
30156   27            Private   Assoc-acdm           12   Married-civ-spouse   
30157   40            Private      HS-grad            9   Married-civ-spouse   
30158   58            Private      HS-grad            9              Widowed   
30159   22            Private      HS-grad            9        Never-married   
30160   52       Self-emp-inc      HS-grad            9   Married-civ-spouse   

               occupation    relationship    race      sex  capitalgain  \
0            Adm-clerical   Not-in-family   White     Male         2174   
1         Exec-managerial         Husband   White     Male            0   
2       Handlers-cleaners   Not-in-family   White     Male            0   
3       Handlers-cleaners         Husband   Black     Male            0   
4          Prof-specialty            Wife   Black   Female            0   
...                   ...             ...     ...      ...          ...   
30156        Tech-support            Wife   White   Female            0   
30157   Machine-op-inspct         Husband   White     Male            0   
30158        Adm-clerical       Unmarried   White   Female            0   
30159        Adm-clerical       Own-child   White     Male            0   
30160     Exec-managerial            Wife   White   Female        15024   

       capitalloss  hoursperweek          native  Salary  
0                0            40   United-States   <=50K  
1                0            13   United-States   <=50K  
2                0            40   United-States   <=50K  
3                0            40   United-States   <=50K  
4                0            40            Cuba   <=50K  
...            ...           ...             ...     ...  
30156            0            38   United-States   <=50K  
30157            0            40   United-States    >50K  
30158            0            40   United-States   <=50K  
30159            0            20   United-States   <=50K  
30160            0            40   United-States    >50K  

[30161 rows x 14 columns]

In [20]:
salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            30161 non-null  int64 
 1   workclass      30161 non-null  object
 2   education      30161 non-null  object
 3   educationno    30161 non-null  int64 
 4   maritalstatus  30161 non-null  object
 5   occupation     30161 non-null  object
 6   relationship   30161 non-null  object
 7   race           30161 non-null  object
 8   sex            30161 non-null  object
 9   capitalgain    30161 non-null  int64 
 10  capitalloss    30161 non-null  int64 
 11  hoursperweek   30161 non-null  int64 
 12  native         30161 non-null  object
 13  Salary         30161 non-null  object
dtypes: int64(5), object(9)
memory usage: 3.2+ MB


In [21]:
salary.isna().sum()

age              0
workclass        0
education        0
educationno      0
maritalstatus    0
occupation       0
relationship     0
race             0
sex              0
capitalgain      0
capitalloss      0
hoursperweek     0
native           0
Salary           0
dtype: int64

In [22]:
salary.describe()

age   educationno   capitalgain   capitalloss  hoursperweek
count  30161.000000  30161.000000  30161.000000  30161.000000  30161.000000
mean      38.438115     10.121316   1092.044064     88.302311     40.931269
std       13.134830      2.550037   7406.466611    404.121321     11.980182
min       17.000000      1.000000      0.000000      0.000000      1.000000
25%       28.000000      9.000000      0.000000      0.000000     40.000000
50%       37.000000     10.000000      0.000000      0.000000     40.000000
75%       47.000000     13.000000      0.000000      0.000000     45.000000
max       90.000000     16.000000  99999.000000   4356.000000     99.000000

In [23]:
x = salary.drop(['workclass','race','capitalloss','capitalgain','Salary'], axis = 1)
y = salary['Salary']

In [24]:
numeric_features = x.select_dtypes(exclude = ['object']).columns
numeric_features

Index(['age', 'educationno', 'hoursperweek'], dtype='object')

In [25]:
categorical_features = x.select_dtypes(include = ['object']).columns
categorical_features
wins = Winsorizer(capping_method = 'iqr', tail = 'both', fold = 1.5, variables = list(numeric_features))
out_pipeline = Pipeline(steps = [('wins', wins)])


In [26]:
outlier_data = out_pipeline.fit_transform(x[numeric_features])


num_pipeline = Pipeline(steps = [('impute', SimpleImputer(strategy = 'mean'))])
num_preprocessor = ColumnTransformer(transformers=[('num', num_pipeline, numeric_features)])
scale_pipeline = Pipeline(steps = [('scale', MinMaxScaler())])


In [27]:
impute_data = num_preprocessor.fit_transform(x[numeric_features])
impute_df = pd.DataFrame(impute_data, columns = numeric_features)

In [28]:
scale_data =scale_pipeline.fit_transform(x[numeric_features])


In [29]:
scale_df = pd.DataFrame(scale_data, columns = numeric_features)
scale_df

age  educationno  hoursperweek
0      0.301370     0.800000      0.397959
1      0.452055     0.800000      0.122449
2      0.287671     0.533333      0.397959
3      0.493151     0.400000      0.397959
4      0.150685     0.800000      0.397959
...         ...          ...           ...
30156  0.136986     0.733333      0.377551
30157  0.315068     0.533333      0.397959
30158  0.561644     0.533333      0.397959
30159  0.068493     0.533333      0.193878
30160  0.479452     0.533333      0.397959

[30161 rows x 3 columns]

In [25]:
from sklearn.preprocessing import OneHotEncoder
encode_pipeline = Pipeline(steps = [('OneHot', OneHotEncoder(drop = 'first'))])
encode_preprocessor = ColumnTransformer(transformers=[('onehot', encode_pipeline, categorical_features)])

In [31]:
encode_data = encode_preprocessor.fit(x[categorical_features])

In [32]:
encode_df = pd.DataFrame((encode_data.transform(x)).toarray())

In [33]:
encode_df.columns = list(encode_preprocessor.get_feature_names_out())
encode_df

onehot__education_ 11th  onehot__education_ 12th  \
0                          0.0                      0.0   
1                          0.0                      0.0   
2                          0.0                      0.0   
3                          1.0                      0.0   
4                          0.0                      0.0   
...                        ...                      ...   
30156                      0.0                      0.0   
30157                      0.0                      0.0   
30158                      0.0                      0.0   
30159                      0.0                      0.0   
30160                      0.0                      0.0   

       onehot__education_ 1st-4th  onehot__education_ 5th-6th  \
0                             0.0                         0.0   
1                             0.0                         0.0   
2                             0.0                         0.0   
3                             0.0                         0.0   
4                             0.0                         0.0   
...                           ...                         ...   
30156                         0.0                         0.0   
30157                         0.0                         0.0   
30158                         0.0                         0.0   
30159                         0.0                         0.0   
30160                         0.0                         0.0   

       onehot__education_ 7th-8th  onehot__education_ 9th  \
0                             0.0                     0.0   
1                             0.0                     0.0   
2                             0.0                     0.0   
3                             0.0                     0.0   
4                             0.0                     0.0   
...                           ...                     ...   
30156                         0.0                     0.0   
30157                         0.0                     0.0   
30158                         0.0                     0.0   
30159                         0.0                     0.0   
30160                         0.0                     0.0   

       onehot__education_ Assoc-acdm  onehot__education_ Assoc-voc  \
0                                0.0                           0.0   
1                                0.0                           0.0   
2                                0.0                           0.0   
3                                0.0                           0.0   
4                                0.0                           0.0   
...                              ...                           ...   
30156                            1.0                           0.0   
30157                            0.0                           0.0   
30158                            0.0                           0.0   
30159                            0.0                           0.0   
30160                            0.0                           0.0   

       onehot__education_ Bachelors  onehot__education_ Doctorate  ...  \
0                               1.0                           0.0  ...   
1                               1.0                           0.0  ...   
2                               0.0                           0.0  ...   
3                               0.0                           0.0  ...   
4                               1.0                           0.0  ...   
...                             ...                           ...  ...   
30156                           0.0                           0.0  ...   
30157                           0.0                           0.0  ...   
30158                           0.0                           0.0  ...   
30159                           0.0                           0.0  ...   
30160                           0.0                           0.0  ...   

       onehot__native_ Portugal  onehot__native_ Puert

In [35]:
cleandata = pd.concat([scale_df, encode_df], axis = 1)
cleandata

age  educationno  hoursperweek  onehot__education_ 11th  \
0      0.301370     0.800000      0.397959                      0.0   
1      0.452055     0.800000      0.122449                      0.0   
2      0.287671     0.533333      0.397959                      0.0   
3      0.493151     0.400000      0.397959                      1.0   
4      0.150685     0.800000      0.397959                      0.0   
...         ...          ...           ...                      ...   
30156  0.136986     0.733333      0.377551                      0.0   
30157  0.315068     0.533333      0.397959                      0.0   
30158  0.561644     0.533333      0.397959                      0.0   
30159  0.068493     0.533333      0.193878                      0.0   
30160  0.479452     0.533333      0.397959                      0.0   

       onehot__education_ 12th  onehot__education_ 1st-4th  \
0                          0.0                         0.0   
1                          0.0                         0.0   
2                          0.0                         0.0   
3                          0.0                         0.0   
4                          0.0                         0.0   
...                        ...                         ...   
30156                      0.0                         0.0   
30157                      0.0                         0.0   
30158                      0.0                         0.0   
30159                      0.0                         0.0   
30160                      0.0                         0.0   

       onehot__education_ 5th-6th  onehot__education_ 7th-8th  \
0                             0.0                         0.0   
1                             0.0                         0.0   
2                             0.0                         0.0   
3                             0.0                         0.0   
4                             0.0                         0.0   
...                           ...                         ...   
30156                         0.0                         0.0   
30157                         0.0                         0.0   
30158                         0.0                         0.0   
30159                         0.0                         0.0   
30160                         0.0                         0.0   

       onehot__education_ 9th  onehot__education_ Assoc-acdm  ...  \
0                         0.0                            0.0  ...   
1                         0.0                            0.0  ...   
2                         0.0                            0.0  ...   
3                         0.0                            0.0  ...   
4                         0.0                            0.0  ...   
...                       ...                            ...  ...   
30156                     0.0                            1.0  ...   
30157                     0.0                            0.0  ...   
30158                     0.0                            0.0  ...   
30159                     0.0                            0.0  ...   
30160                     0.0                            0.0  ...   

       onehot__native_ Portugal  onehot__native_ Puerto-Rico  \
0                           0.0                          0.0   
1                           0.0                          0.0   
2                           0.0                          0.0   
3                           0.0                          0.0   
4                           0.0                          0.0   
...                         ...                          ...   
30156                       0.0                          0.0   
30157                       0.0                          0.0   
30158                       0.0                          0.0   
30159                       0.0                          0.0   
30160                       0.0                          0.0   

       onehot__native_ Scotland  on

In [37]:
joblib.dump(impute_data, 'SVM_assignment_impute')
joblib.dump(outlier_data, 'SVM_outlier_assignment')
joblib.dump(scale_data, 'SVM_assignment_scale')
joblib.dump(encode_data, 'SVM_assignment_encode')

['SVM_assignment_encode']

In [38]:
#model building

In [44]:
x_train, x_test, y_train, y_test = train_test_split(cleandata, y, test_size = 0.2, random_state = 42)

In [40]:
model_linear = SVC(kernel = 'linear')


In [45]:
model1 = model_linear.fit(x_train, y_train)

In [46]:
pred_test =model1.predict(x_test)

In [47]:
np.mean(pred_test == y_test)

0.8238024200232057

In [48]:
### Hyperparameter Optimization


In [49]:
model = SVC()

In [50]:
parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['linear', 'poly','rbf','sigmoid']}

In [51]:
# Randomized Search Technique for exhaustive search for best model


In [54]:
rand_search =RandomizedSearchCV(model, parameters, n_iter = 10, n_jobs = 3, cv = 3, scoring = 'accuracy', random_state =0)

In [57]:
r = rand_search.fit(x_train, y_train)

In [58]:
r.best_params_

{'kernel': 'rbf', 'gamma': 0.001, 'C': 100}

In [62]:
best = r.best_estimator_

In [60]:
r_pred = r.predict(x_test)

In [61]:
np.mean(r_pred == y_test)

0.8251284601359191

In [64]:
pickle.dump(best, open('SVM_Assignment_model_best.pkl','wb') )

In [65]:
#prediction for test data

In [15]:
test = pd.read_csv('SalaryData_Test.csv')

In [68]:
impute =joblib.load('SVM_assignment_impute')
winzor = joblib.load('SVM_outlier_assignment')
minmax = joblib.load('SVM_assignment_scale')


In [31]:
import pickle

with open('SVM_Assignment_model_best.pkl', 'rb') as file:
    model2 = pickle.load(file)


In [17]:
test1 =test.drop(['workclass','race','capitalloss','capitalgain','Salary'], axis = 1)
numeric_features = test1.select_dtypes(exclude = 'object').columns
categorical_features = test1.select_dtypes(include = 'object').columns

In [18]:
impute1 = SimpleImputer(strategy = 'mean')
impute_data1 = impute1.fit(test1[numeric_features])

In [20]:
wins1 = Winsorizer(capping_method  = 'gaussian', tail = 'both', fold = 1.5, variables = list(test1[numeric_features]))
wins1_d = wins1.fit(test1[numeric_features])

In [21]:
cleandata1 = pd.DataFrame(impute1.transform(test1[numeric_features]), columns = numeric_features)

In [22]:
cleandata2 = pd.DataFrame(wins1.transform(cleandata1), columns = numeric_features)

In [23]:
scale1 = MinMaxScaler()
scale1_d = scale1.fit(cleandata2)

In [47]:
encode_pipeline1 = Pipeline(steps = [('OneHot', OneHotEncoder(drop = 'first'))])
encode_preprocessor1 = ColumnTransformer(transformers=[('onehot', encode_pipeline1, categorical_features)])
encode1_d = encode_preprocessor1.fit(test1[categorical_features])
encode1_df = pd.DataFrame(encode1_d.transform(test1[categorical_features]).toarray(), columns= encode_preprocessor1.get_feature_names_out(categorical_features))

cleandata3 = pd.DataFrame(scale1_d.transform(cleandata2), columns = numeric_features)

In [48]:
cleandata4 = pd.concat([cleandata3, encode1_df], axis = 1)


In [50]:
prediction = pd.DataFrame(model2.predict(cleandata4), columns = ['choice_pred'])

In [52]:
final = pd.concat([cleandata4, prediction], axis = 1)
final

age  educationno  hoursperweek  onehot__education_ 11th  \
0      0.156998     0.094479      0.473704                      1.0   
1      0.480859     0.355034      0.750044                      0.0   
2      0.231735     0.745866      0.473704                      0.0   
3      0.630333     0.485311      0.473704                      0.0   
4      0.381210     0.000000      0.197364                      0.0   
...         ...          ...           ...                      ...   
15055  0.356297     0.876144      0.473704                      0.0   
15056  0.505772     0.876144      0.363168                      0.0   
15057  0.480859     0.876144      0.750044                      0.0   
15058  0.630333     0.876144      0.473704                      0.0   
15059  0.406122     0.876144      1.000000                      0.0   

       onehot__education_ 12th  onehot__education_ 1st-4th  \
0                          0.0                         0.0   
1                          0.0                         0.0   
2                          0.0                         0.0   
3                          0.0                         0.0   
4                          0.0                         0.0   
...                        ...                         ...   
15055                      0.0                         0.0   
15056                      0.0                         0.0   
15057                      0.0                         0.0   
15058                      0.0                         0.0   
15059                      0.0                         0.0   

       onehot__education_ 5th-6th  onehot__education_ 7th-8th  \
0                             0.0                         0.0   
1                             0.0                         0.0   
2                             0.0                         0.0   
3                             0.0                         0.0   
4                             0.0                         0.0   
...                           ...                         ...   
15055                         0.0                         0.0   
15056                         0.0                         0.0   
15057                         0.0                         0.0   
15058                         0.0                         0.0   
15059                         0.0                         0.0   

       onehot__education_ 9th  onehot__education_ Assoc-acdm  ...  \
0                         0.0                            0.0  ...   
1                         0.0                            0.0  ...   
2                         0.0                            1.0  ...   
3                         0.0                            0.0  ...   
4                         0.0                            0.0  ...   
...                       ...                            ...  ...   
15055                     0.0                            0.0  ...   
15056                     0.0                            0.0  ...   
15057                     0.0                            0.0  ...   
15058                     0.0                            0.0  ...   
15059                     0.0                            0.0  ...   

       onehot__native_ Puerto-Rico  onehot__native_ Scotland  \
0                              0.0                       0.0   
1                              0.0                       0.0   
2                              0.0                       0.0   
3                              0.0                       0.0   
4                              0.0                       0.0   
...                            ...                       ...   
15055                          0.0                       0.0   
15056                          0.0                       0.0   
15057                          0.0                       0.0   
15058                          0.0                       0.0   
15059                          0.0                       0.0   

       onehot__native_ South  oneho